In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [2]:
###input
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter_add_no_ec.csv'

target_smiles_path = '../../../Results/lipid/target_smiles_lipid.pickle'
target_model_smiles_path = '../../../Results/lipid/yeast_met_lipid.pickle'
yeast8_metabolites_path = '../../../Data/model/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'

cut_off = 0.3
num = 'all'
num_process = 5
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re/'
sce_gene_clean_ec = '../../../Results/ECnumber_prediction/Saccharomyces_cerevisiae_teacher_maxsep.csv'
DeepEC_path = '../../../Results/ECnumber_prediction/Sce_DeepECv2.txt'
sce_uniprot_path = '../../../Data/uniprot/uniprotkb_organism_id_559292_2023_11_08.tsv'

yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'
rxndb_to_model_with_target_success_path = cut_off_path + 'rxndb_to_model_with_target_topall_0.3_success.csv'
metabolites_info_to_GEM_path_not_lipid_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_top50_0.3.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'


###output

rxndb_to_model_add_ec_path = cut_off_path + f'rxndb_to_model_add_ec_top{num}_{cut_off}.csv'
rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}.csv'
rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}.csv'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'



In [3]:
def process_smiles_annotation(i,yeast_total_smiles):    ###The highest similarity scores of all the reactants and model metabolites of rxn were obtained
    result = {'smile': i, 'score': 0, 'sim_smile': ''}
    score_init = 0
    sim_smile = ' '
    for j in yeast_total_smiles:
        # tmp_score = compare_smiles(i, j)  
        if compare_smiles_inchikey(i,j) :
            score_init = 1
            sim_smile = j
            break
    result['score'] = score_init
    result['sim_smile'] = sim_smile
    return result
def calculate_smile_max_score_annotation(rxndb_all_reactant_smile,yeast_total_smiles,num_process=30):
    smile_max_score = {'smile': [], 'score': [], 'sim_smile': []}

    pool = multiprocessing.Pool(num_process)

    partial_process_smiles = partial(process_smiles_annotation,yeast_total_smiles=yeast_total_smiles)
    results = list(tqdm(pool.imap(partial_process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))
    # results = list(tqdm(pool.imap(process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))

    pool.close()
    pool.join()

    for result in results:
        smile_max_score['smile'].append(result['smile'])
        smile_max_score['score'].append(result['score'])
        smile_max_score['sim_smile'].append(result['sim_smile'])
        
    return(smile_max_score)

In [ ]:
#run find_no_ec_reaction_lipid_re.ipynb to get the rxndb_to_model_with_target_topall_0.3_success.csv file
data_add_no_ec = pd.read_csv(rxndb_to_model_with_target_success_path)
data_no_ec = data_add_no_ec[data_add_no_ec['EC number'].isnull()]
data_ec = data_add_no_ec[~data_add_no_ec['EC number'].isnull()]

In [ ]:
data = pd.read_csv(rxndb_to_model_path)
print('data:', data.shape)

print('Unique NO values in data before merge:', data['NO'].nunique())

data = pd.concat([data, data_ec])


data = data.drop(columns=['NO'])

data = data.reset_index(drop=True)
data.index.name = 'NO'
data = data.reset_index()

print( data['NO'].nunique())
data.to_csv(rxndb_to_model_add_ec_path,index=None)


In [6]:
model = cobra.io.load_yaml_model(yeast870_path)

model_genes = model.genes

model_genes = [x.id for x in model_genes]

### CLEAN

In [7]:
clean_gene2ec_dict = get_gene2ec_dict_clean(sce_gene_clean_ec)

### DeepECv2

In [8]:
DeepEC_gene2ec_dict = get_gene2ec_dict_DeepEC(DeepEC_path)

In [ ]:
clean_deepec_gene2ec_dict = {}
for k,v in clean_gene2ec_dict.items():
    if k in DeepEC_gene2ec_dict or k in model_genes:
        clean_deepec_gene2ec_dict[k] = v
clean_deepec_ec2gene_dict = get_ec2gene_dict_clean(clean_deepec_gene2ec_dict)
print(len(clean_deepec_ec2gene_dict))

In [ ]:
rxndb_with_GPR = rxndb_gene_annotation(rxndb_to_model_add_ec_path,clean_deepec_ec2gene_dict)
rxndb_with_GPR = rxndb_with_GPR[rxndb_with_GPR['GENE'].apply(lambda x: len(x) > 0)]
data_no_ec['GENE'] = None
data_no_ec['GENE'] = data_no_ec['GENE'].apply(lambda x: [])
data_no_ec['EC number'] = data_no_ec['EC number'].apply(lambda x: [])
print('rxndb with GPR:',rxndb_with_GPR.shape)

In [ ]:
print('rxndb_with_GPR:',rxndb_with_GPR.shape)

rxndb_with_GPR = pd.concat([rxndb_with_GPR,data_no_ec])
rxndb_with_GPR = rxndb_with_GPR.drop(columns=['NO'])
rxndb_with_GPR = rxndb_with_GPR.reset_index(drop=True)
rxndb_with_GPR.index.name = 'NO'
rxndb_with_GPR = rxndb_with_GPR.reset_index()

print( rxndb_with_GPR['NO'].nunique())
# rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)

In [ ]:
rxndb_with_GPR['NO'] = rxndb_with_GPR['NO'].apply(lambda x: f'rxnl{x}')
rxndb_with_GPR.head()


In [13]:
rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)


### rxndb metabolite annotation

In [ ]:
#rxndb all metabolites
rxndb_all_smile = get_all_met_smile(rxndb_GPR_to_model_path)


In [ ]:
#metabolite comparison
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
smile_max_score = calculate_smile_max_score_annotation(rxndb_all_smile,yeast8_total_smiles,num_process=num_process)
smile_max_score_pd = pd.DataFrame(smile_max_score)
smile_max_score_pd.to_csv(rxndb_met_max_score_met_annotation_file,index=None)


In [ ]:
#assign ID
yeast8_id_smiles_mapping = get_yeast8_id_smiles_mapping(yeast8_metabolites_path)
smile_max_score_pd_with_ID = assign_ID_to_RXNDB_met(smile_max_score_pd,yeast8_id_smiles_mapping,s = 'lipid')
metabolites_info_to_GEM = get_new_met_smile_list(smile_max_score_pd_with_ID)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)



In [ ]:
metabolites_info_to_GEM_path_not_lipid = pd.read_csv(metabolites_info_to_GEM_path_not_lipid_path)
metabolites_info_to_GEM_path_not_lipid['inchikey0'] = metabolites_info_to_GEM_path_not_lipid['new_met_smiles'].apply(smiles2inchikey0)
for i in tqdm(range(metabolites_info_to_GEM.shape[0]),total=metabolites_info_to_GEM.shape[0]):
    if 'sl_' not in metabolites_info_to_GEM['ID'][i]:
        continue
    if smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i]) in metabolites_info_to_GEM_path_not_lipid['inchikey0'].values:
        metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)

### Generate equation according to metabolite ID

In [ ]:
smiles_to_id_mapping = get_smiles_to_id_mapping(metabolites_info_to_GEM_path)
print(len(smiles_to_id_mapping))

In [ ]:
rxndb_total_info_to_model = get_rxndb_to_model_total_info(rxndb_GPR_to_model_path,smiles_to_id_mapping)
rxndb_total_info_to_model = rxndb_total_info_to_model[['NO','EC_number','templateID','rxn_smiles_basic','rxn_smiles_final','GPR','equation_dict','equation']]
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

### check the formula of the model metabolites in the predicted reaction

In [ ]:
metabolites_info_to_GEM,rxndb_total_info_to_model = check_met_and_reaction_formula(yeast8_metabolites_path,metabolites_info_to_GEM_path,rxndb_total_info_to_model_path)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

### remove duplicate reactions and existing reactions

In [ ]:
same_rxndb_reaction = get_same_rxndb_reaction_mapping(rxndb_total_info_to_model_path)
rxndb_total_info_to_model = get_no_same_rxndb_reaction_parallelize(rxndb_total_info_to_model_path,get_no_same_rxndb_reaction,same_rxndb_reaction,n_cores=num_process)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

In [25]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
rxndb_to_model_total_info = pd.read_csv(rxndb_total_info_to_model_path)


### Find out the reactions that already exist in the model

In [26]:
with open(miss_met_id_file) as f:
    miss_met_id = json.load(f) 
model = cobra.io.load_yaml_model(yeast870_path)


### yeast8_smiles_list, ID with smiles

In [ ]:
model_excel, metID_with_smiles, yeast8_smiles_lst = process_model(yeast8_metabolites_path, miss_met_id)


In [ ]:
rxndb_met_with_s_smiles_lst = get_rxndb_met_with_s_smiles_lst(rxndb_total_info_to_model_path,model_excel,miss_met_id)

reaID_with_smiles_dict = get_reaID_with_smiles_dict(model, model_excel, miss_met_id,rxndb_met_with_s_smiles_lst,metID_with_smiles)


In [ ]:
yeast8_reaction_in_rxndb = get_yeast8_reaction_in_rxndb_parallel(rxndb_total_info_to_model_path,reaID_with_smiles_dict,model_excel,miss_met_id,num_process=num_process)


In [30]:
with open(yeast8_reaction_in_rxndb_json, 'w') as json_file:
    json.dump(yeast8_reaction_in_rxndb, json_file,indent=4)

In [ ]:
rxndb_to_model_total_info['equation_dict'] = rxndb_to_model_total_info['equation_dict'].apply(lambda x:literal_eval(x))
print(rxndb_to_model_total_info.shape)
# rxndb_to_model_total_info['equation_dict'] = rxndb_to_model_total_info['equation_dict'].apply(lambda x:{k:v for k, v in x.items() if k not in miss_met_id})
# rxndb_to_model_total_info = rxndb_to_model_total_icnfo[~rxndb_to_model_total_info['equation_dict'].apply(lambda x : all('s_' in k for k, v in x.items()))]
rxndb_to_model_total_info = rxndb_to_model_total_info[rxndb_to_model_total_info['equation_dict'].apply(lambda x : any('sl_' in k for k, v in x.items()))]
print(rxndb_to_model_total_info.shape)
# rxndb_to_model_total_info = rxndb_to_model_total_info[~rxndb_to_model_total_info['equation_dict'].apply(lambda x : all('sn_' in k for k, v in x.items()))]
# print(rxndb_to_model_total_info.shape)

In [32]:
rxndb_to_model_total_info.to_csv(rxndb_total_info_to_model_path,index=None)
